In [137]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_together import ChatTogether
from langchain_core.output_parsers import JsonOutputParser
import json
from pydantic import BaseModel, Field

from dotenv import load_dotenv
import os

In [135]:
%dotenv "../.env"

UsageError: Line magic function `%dotenv` not found.


In [136]:
load_dotenv()

True

In [130]:

react_js_interview_question = [
    "What is React?",
    "What are the key features of React?",
    "What is JSX?",
    "What is the virtual DOM?",
    "What is a component in React?",
    "What is the difference between state and props in React?",
    "What are React hooks?",
    "What is the significance of keys in React?",
    "What is the purpose of render() in React?",
    "What is the use of ref in React?",
    "What are the lifecycle methods in React?",
    "What is the significance of context in React?",
    "What is the significance of portals in React?",
    "What are higher-order components in React?",
    "What is the significance of error boundaries in React?",
    "What is the significance of fragments in React?",
    "What is the significance of lazy loading in React?",
]

react_js_interview_expected_answers = [
    "React is a JavaScript library used for building user interfaces.",
    "Key features of React include JSX, components, virtual DOM, React hooks, and more.",
    "JSX is a syntax extension for JavaScript that allows HTML to be written in JavaScript code.",
    "The virtual DOM is a lightweight copy of the actual DOM that React uses to improve performance.",
    "A component is a reusable piece of code that represents a part of the user interface.",
    "State is mutable data that is managed by a component, while props are immutable data passed from a parent component.",
    "React hooks are functions that let you use state and other React features in functional components.",
    "Keys are used to uniquely identify elements in a list and improve performance.",
    "The render() method is used to render the component's UI based on its current state and props.",
    "The ref attribute is used to reference a React element or component.",
    "Lifecycle methods are methods that are called at different stages of a component's life cycle.",
    "Context provides a way to pass data through the component tree without having to pass props down manually at every level.",
    "Portals provide a way to render children components outside the DOM hierarchy of the parent component.",
    "Higher-order components are functions that take a component and return a new component with additional functionality.",
    "Error boundaries are React components that catch JavaScript errors anywhere in their child component tree.",
    "Fragments allow you to group multiple elements without adding extra nodes to the DOM.",
    "Lazy loading is a technique that defers the loading of non-essential resources until they are needed.",
]

qa_template = """
You are an AI designed to evaluate the quality of a user's answer based on the question provided for a React JS interview.


Question: "{question}"

User's answer: "{user_answer}"

Rate the user's answer on a scale from 1 to 10 based on its relevance, accuracy, and completeness (with 1 being completely irrelevant or incorrect, and 10 being a perfect answer).
{format_instructions}
"""
# Only return your output as json strong with the following key:
# - rating
compare_template = """
You are an AI designed to evaluate the quality of a user's answer based on the provided expected answer.

Expected answer: "{expected_answer}"

User's answer: "{user_answer}"

Rate the user's answer on a scale from 1 to 10 based on its relevance (with 1 being completely irrelevant or incorrect, and 10 being a perfect answer).Do not be very strict
Return your output in JSON format with the following keys:
- rating

"""


class RatingModel(BaseModel):
    rating: int = Field(..., ge=1, le=10)


parser = JsonOutputParser(pydantic_object=RatingModel)

qa_prompt = PromptTemplate(input_variables=["question", "user_answer"], template=qa_template, partial_variables={
                           "format_instructions": parser.get_format_instructions()})
compare_prompt = PromptTemplate(
    input_variables=["expected_answer", "user_answer"], template=compare_template)


prompt_templates = {
    "qa": qa_prompt,
    "compare": compare_prompt
}


class LLM_Api:
    def __init__(self, model_name, temperature=0.0, api_key=None, chatbot_type="qa"):

        if api_key is None:
            raise ValueError("API key is required.")
        try:
            llm = ChatTogether(
                api_key=api_key, temperature=temperature, model=model_name)
            self.llm_chain = prompt_templates[chatbot_type] | llm | parser
        except Exception as e:
            raise ValueError("Error initializing the LLM chain: " + str(e))

    def rate_answer(self, question, user_answer):
        return self.llm_chain.invoke({"question": question, "user_answer": user_answer})

    def compare_answers(self, expected_answer, user_answer):
        return self.llm_chain.invoke({"expected_answer": expected_answer, "user_answer": user_answer})

In [127]:
api = LLM_Api(model_name="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo", temperature=0.0,
              api_key=os.environ["TOGETHER_AI_API_KEY"], chatbot_type="qa")

In [ ]:
rating = api.rate_answer(react_js_interview_question[0], "React is a JavaScript library for building user interfaces.")

In [123]:
# rating = rate_answer_langchain_2("React is a JavaScript library for building user interfaces.", "It is a web based javascript library for building websites.")

In [128]:
rating

{'rating': 8}

In [1]:
import numpy as np

In [14]:
a=[ 330, 11500, 16500, 914, 796, 353, 6470, 219, 389, 805, 256, 919, 398, 900, 496, -6240, 948, 711, 306, 777, 719, 545, 248]
a=np.array(a)
b=np.abs(np.array(a) -  np.mean(a))
b
mask = b > 2 * np.std(a)
mask
a[mask] = np.median(a)

array([ 1376.91304348,  9793.08695652, 14793.08695652,   792.91304348,
         910.91304348,  1353.91304348,  4763.08695652,  1487.91304348,
        1317.91304348,   901.91304348,  1450.91304348,   787.91304348,
        1308.91304348,   806.91304348,  1210.91304348,  7946.91304348,
         758.91304348,   995.91304348,  1400.91304348,   929.91304348,
         987.91304348,  1161.91304348,  1458.91304348])

In [13]:
a

array([  330,   711,   711,   914,   796,   353,  6470,   219,   389,
         805,   256,   919,   398,   900,   496, -6240,   948,   711,
         306,   777,   719,   545,   248])